In [51]:
import pandas as pd
import re
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve, auc

In [55]:
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout

In [48]:
PATH = '~/data/toxic/data/'

train = pd.read_csv(PATH + 'cleaned_train.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')

#train_sentence = train['comment_text_cleaned']
#test_sentence = test['comment_text_cleaned']

#text = pd.concat([train_sentence, test_sentence])

#print(train.shape)
print(test.shape)

(153164, 21)


In [26]:
submission = pd.read_csv('/home/kai/data/shiyi/toxic/submissions/sub_FastText_cleaned_200_300_50_50_0.1_32_4_1518806221.csv')

In [27]:
submission.head(2)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999523,4.868310e-01,0.986272,1.233855e-01,9.359576e-01,5.140475e-01
1,0000247867823ef7,0.000037,3.082599e-10,0.000001,6.009321e-08,6.191491e-07,4.219558e-07


In [28]:
sum(test.id == submission.id) == test.shape[0]

True

In [29]:
test[['id','comment_text_cleaned','word_count']].head(2) # test data from csv

,id,comment_text_cleaned,word_count
0,00001cee341fdb12,yo bitch ja rule be more succesful then you wi...,72
1,0000247867823ef7,from rfc the title be fine a it be imo .,12


In [30]:
toxic_words_poll = ['motherfucker', 'crap', 'fuck', 'prick', 'piss', 'hell', 'nerd', 'life', 'sick', 'jerk', 'racist', 'goddamn', 'wtf', 'shove', 'sucker', 'dude', 'motherfucking', 'mom', 'dare', 'stop', 'filthy', 'ignorant', 'fuckhead', 'youre', 'stupid', 'gonna', 'disgust', 'kiss', 'yo', 'get', 'pathetic', 'stupidity', 'liar', 'queer', 'fuk', 'bitch', 'retard', 'arsehole', 'screw', 'turd', 'suck', 'little', 'anus', 'stink', 'hole', 'dick', 'fuckwit', 'fucken', 'quit', 'scumbag', 'asshole', 'wank', 'fucktard', 'smelly', 'idiotic', 'jackass', 'sad', 'mother', 'wanna', 'disgrace', 'ya', 'cum', 'maggot', 'basement', 'lame', 'idiot', 'nazi', 'bunch', 'cking', 'ha', 'dumb', 'ban', 'go', 'hypocrite', 'commie', 'monkey', 'arrogant', 'butt', 'garbage', 'burn', 'homosexual', 'cant', 'cunt', 'stfu', 'douchebag', 'fuckface', 'bastard', 'cuz', 'stinky', 'masturbate', 'mommy', 'thats', 'clown', 'ever', 'hoe', 'pervert', 'man', 'laugh', 'bully', 'scumbags']
print(len(toxic_words_poll))

100


In [31]:
toxic_words_poll_reordered = ['motherfucking', 'fuckhead', 'fuckface', 'motherfucker', 'fucken', 'fucktard', 'fuck', 'fuckwit', 'asshole', 'bitch', 'sucker', 'cunt', 'arsehole', 'suck', 'stinky', 'scumbags', 'smelly', 'bastard', 'goddamn', 'maggot', 'wank', 'douchebag', 'fuk', 'scumbag', 'dick', 'retard', 'prick', 'filthy', 'idiot', 'masturbate', 'jackass', 'dumb', 'queer', 'stupid', 'nerd', 'cking', 'shove', 'jerk', 'stink', 'mommy', 'youre', 'wtf', 'piss', 'pathetic', 'hypocrite', 'commie', 'stfu', 'mother', 'basement', 'stupidity', 'kiss', 'idiotic', 'pervert', 'clown', 'crap', 'disgust', 'disgrace', 'cuz', 'sick', 'homosexual', 'screw', 'racist', 'gonna', 'garbage', 'monkey', 'ignorant', 'wanna', 'dare', 'dude', 'arrogant', 'nazi', 'anus', 'bunch', 'hole', 'burn', 'bully', 'life', 'turd', 'thats', 'laugh', 'butt', 'lame', 'stop', 'little', 'hoe', 'mom', 'sad', 'hell', 'yo', 'liar', 'get', 'ban', 'go', 'ya', 'cum', 'cant', 'ever', 'quit', 'ha', 'man']

In [32]:
step_roc_dict = ['step<0.01>_words<1>_length<150>', 'step<0.01>_words<1>_length<200>', 'step<0.01>_words<1>_length<250>', 'step<0.01>_words<2>_length<150>', 'step<0.01>_words<2>_length<200>', 'step<0.01>_words<2>_length<250>', 'step<0.01>_words<3>_length<150>', 'step<0.01>_words<3>_length<200>', 'step<0.01>_words<3>_length<250>', 'step<0.01>_words<4>_length<150>', 'step<0.01>_words<4>_length<200>', 'step<0.01>_words<4>_length<250>', 'step<0.01>_words<5>_length<150>', 'step<0.01>_words<5>_length<200>', 'step<0.01>_words<5>_length<250>', 'step<0.02>_words<1>_length<150>', 'step<0.02>_words<1>_length<200>', 'step<0.02>_words<1>_length<250>', 'step<0.02>_words<2>_length<150>', 'step<0.02>_words<2>_length<200>', 'step<0.02>_words<2>_length<250>', 'step<0.02>_words<3>_length<150>', 'step<0.02>_words<3>_length<200>', 'step<0.02>_words<3>_length<250>', 'step<0.02>_words<4>_length<150>', 'step<0.02>_words<4>_length<200>', 'step<0.02>_words<4>_length<250>', 'step<0.02>_words<5>_length<150>', 'step<0.02>_words<5>_length<200>', 'step<0.02>_words<5>_length<250>', 'step<0.03>_words<1>_length<150>', 'step<0.03>_words<1>_length<200>', 'step<0.03>_words<1>_length<250>', 'step<0.03>_words<2>_length<150>', 'step<0.03>_words<2>_length<200>', 'step<0.03>_words<2>_length<250>', 'step<0.03>_words<3>_length<150>', 'step<0.03>_words<3>_length<200>', 'step<0.03>_words<3>_length<250>', 'step<0.03>_words<4>_length<150>', 'step<0.03>_words<4>_length<200>', 'step<0.03>_words<4>_length<250>', 'step<0.03>_words<5>_length<150>', 'step<0.03>_words<5>_length<200>', 'step<0.03>_words<5>_length<250>', 'step<0.04>_words<1>_length<150>', 'step<0.04>_words<1>_length<200>', 'step<0.04>_words<1>_length<250>', 'step<0.04>_words<2>_length<150>', 'step<0.04>_words<2>_length<200>', 'step<0.04>_words<2>_length<250>', 'step<0.04>_words<3>_length<150>', 'step<0.04>_words<3>_length<200>', 'step<0.04>_words<3>_length<250>', 'step<0.04>_words<4>_length<150>', 'step<0.04>_words<4>_length<200>', 'step<0.04>_words<4>_length<250>', 'step<0.04>_words<5>_length<150>', 'step<0.04>_words<5>_length<200>', 'step<0.04>_words<5>_length<250>', 'step<0.05>_words<1>_length<150>', 'step<0.05>_words<1>_length<200>', 'step<0.05>_words<1>_length<250>', 'step<0.05>_words<2>_length<150>', 'step<0.05>_words<2>_length<200>', 'step<0.05>_words<2>_length<250>', 'step<0.05>_words<3>_length<150>', 'step<0.05>_words<3>_length<200>', 'step<0.05>_words<3>_length<250>', 'step<0.05>_words<4>_length<150>', 'step<0.05>_words<4>_length<200>', 'step<0.05>_words<4>_length<250>', 'step<0.05>_words<5>_length<150>', 'step<0.05>_words<5>_length<200>', 'step<0.05>_words<5>_length<250>', 'step<0.06>_words<1>_length<150>', 'step<0.06>_words<1>_length<200>', 'step<0.06>_words<1>_length<250>', 'step<0.06>_words<2>_length<150>', 'step<0.06>_words<2>_length<200>', 'step<0.06>_words<2>_length<250>', 'step<0.06>_words<3>_length<150>', 'step<0.06>_words<3>_length<200>', 'step<0.06>_words<3>_length<250>', 'step<0.06>_words<4>_length<150>', 'step<0.06>_words<4>_length<200>', 'step<0.06>_words<4>_length<250>', 'step<0.06>_words<5>_length<150>', 'step<0.06>_words<5>_length<200>', 'step<0.06>_words<5>_length<250>', 'step<0.07>_words<1>_length<150>', 'step<0.07>_words<1>_length<200>', 'step<0.07>_words<1>_length<250>', 'step<0.07>_words<2>_length<150>', 'step<0.07>_words<2>_length<200>', 'step<0.07>_words<2>_length<250>', 'step<0.07>_words<3>_length<150>', 'step<0.07>_words<3>_length<200>', 'step<0.07>_words<3>_length<250>', 'step<0.07>_words<4>_length<150>', 'step<0.07>_words<4>_length<200>', 'step<0.07>_words<4>_length<250>', 'step<0.07>_words<5>_length<150>', 'step<0.07>_words<5>_length<200>', 'step<0.07>_words<5>_length<250>', 'step<0.08>_words<1>_length<150>', 'step<0.08>_words<1>_length<200>', 'step<0.08>_words<1>_length<250>', 'step<0.08>_words<2>_length<150>', 'step<0.08>_words<2>_length<200>', 'step<0.08>_words<2>_length<250>', 'step<0.08>_words<3>_length<150>', 'step<0.08>_words<3>_length<200>', 'step<0.08>_words<3>_length<250>', 'step<0.08>_words<4>_length<150>', 'step<0.08>_words<4>_length<200>', 'step<0.08>_words<4>_length<250>', 'step<0.08>_words<5>_length<150>', 'step<0.08>_words<5>_length<200>', 'step<0.08>_words<5>_length<250>', 'step<0.09>_words<1>_length<150>', 'step<0.09>_words<1>_length<200>', 'step<0.09>_words<1>_length<250>', 'step<0.09>_words<2>_length<150>', 'step<0.09>_words<2>_length<200>', 'step<0.09>_words<2>_length<250>', 'step<0.09>_words<3>_length<150>', 'step<0.09>_words<3>_length<200>', 'step<0.09>_words<3>_length<250>', 'step<0.09>_words<4>_length<150>', 'step<0.09>_words<4>_length<200>', 'step<0.09>_words<4>_length<250>', 'step<0.09>_words<5>_length<150>', 'step<0.09>_words<5>_length<200>', 'step<0.09>_words<5>_length<250>', 'step<0.1>_words<1>_length<150>', 'step<0.1>_words<1>_length<200>', 'step<0.1>_words<1>_length<250>', 'step<0.1>_words<2>_length<150>', 'step<0.1>_words<2>_length<200>', 'step<0.1>_words<2>_length<250>', 'step<0.1>_words<3>_length<150>', 'step<0.1>_words<3>_length<200>', 'step<0.1>_words<3>_length<250>', 'step<0.1>_words<4>_length<150>', 'step<0.1>_words<4>_length<200>', 'step<0.1>_words<4>_length<250>', 'step<0.1>_words<5>_length<150>', 'step<0.1>_words<5>_length<200>', 'step<0.1>_words<5>_length<250>', 'step<0.11>_words<1>_length<150>', 'step<0.11>_words<1>_length<200>', 'step<0.11>_words<1>_length<250>', 'step<0.11>_words<2>_length<150>', 'step<0.11>_words<2>_length<200>', 'step<0.11>_words<2>_length<250>', 'step<0.11>_words<3>_length<150>', 'step<0.11>_words<3>_length<200>', 'step<0.11>_words<3>_length<250>', 'step<0.11>_words<4>_length<150>', 'step<0.11>_words<4>_length<200>', 'step<0.11>_words<4>_length<250>', 'step<0.11>_words<5>_length<150>', 'step<0.11>_words<5>_length<200>', 'step<0.11>_words<5>_length<250>', 'step<0.12>_words<1>_length<150>', 'step<0.12>_words<1>_length<200>', 'step<0.12>_words<1>_length<250>', 'step<0.12>_words<2>_length<150>', 'step<0.12>_words<2>_length<200>', 'step<0.12>_words<2>_length<250>', 'step<0.12>_words<3>_length<150>', 'step<0.12>_words<3>_length<200>', 'step<0.12>_words<3>_length<250>', 'step<0.12>_words<4>_length<150>', 'step<0.12>_words<4>_length<200>', 'step<0.12>_words<4>_length<250>', 'step<0.12>_words<5>_length<150>', 'step<0.12>_words<5>_length<200>', 'step<0.12>_words<5>_length<250>', 'step<0.13>_words<1>_length<150>', 'step<0.13>_words<1>_length<200>', 'step<0.13>_words<1>_length<250>', 'step<0.13>_words<2>_length<150>', 'step<0.13>_words<2>_length<200>', 'step<0.13>_words<2>_length<250>', 'step<0.13>_words<3>_length<150>', 'step<0.13>_words<3>_length<200>', 'step<0.13>_words<3>_length<250>', 'step<0.13>_words<4>_length<150>', 'step<0.13>_words<4>_length<200>', 'step<0.13>_words<4>_length<250>', 'step<0.13>_words<5>_length<150>', 'step<0.13>_words<5>_length<200>', 'step<0.13>_words<5>_length<250>', 'step<0.14>_words<1>_length<150>', 'step<0.14>_words<1>_length<200>', 'step<0.14>_words<1>_length<250>', 'step<0.14>_words<2>_length<150>', 'step<0.14>_words<2>_length<200>', 'step<0.14>_words<2>_length<250>', 'step<0.14>_words<3>_length<150>', 'step<0.14>_words<3>_length<200>', 'step<0.14>_words<3>_length<250>', 'step<0.14>_words<4>_length<150>', 'step<0.14>_words<4>_length<200>', 'step<0.14>_words<4>_length<250>', 'step<0.14>_words<5>_length<150>', 'step<0.14>_words<5>_length<200>', 'step<0.14>_words<5>_length<250>', 'step<0.15>_words<1>_length<150>', 'step<0.15>_words<1>_length<200>', 'step<0.15>_words<1>_length<250>', 'step<0.15>_words<2>_length<150>', 'step<0.15>_words<2>_length<200>', 'step<0.15>_words<2>_length<250>', 'step<0.15>_words<3>_length<150>', 'step<0.15>_words<3>_length<200>', 'step<0.15>_words<3>_length<250>', 'step<0.15>_words<4>_length<150>', 'step<0.15>_words<4>_length<200>', 'step<0.15>_words<4>_length<250>', 'step<0.15>_words<5>_length<150>', 'step<0.15>_words<5>_length<200>', 'step<0.15>_words<5>_length<250>', 'step<0.16>_words<1>_length<150>', 'step<0.16>_words<1>_length<200>', 'step<0.16>_words<1>_length<250>', 'step<0.16>_words<2>_length<150>', 'step<0.16>_words<2>_length<200>', 'step<0.16>_words<2>_length<250>', 'step<0.16>_words<3>_length<150>', 'step<0.16>_words<3>_length<200>', 'step<0.16>_words<3>_length<250>', 'step<0.16>_words<4>_length<150>', 'step<0.16>_words<4>_length<200>', 'step<0.16>_words<4>_length<250>', 'step<0.16>_words<5>_length<150>', 'step<0.16>_words<5>_length<200>', 'step<0.16>_words<5>_length<250>', 'step<0.17>_words<1>_length<150>', 'step<0.17>_words<1>_length<200>', 'step<0.17>_words<1>_length<250>', 'step<0.17>_words<2>_length<150>', 'step<0.17>_words<2>_length<200>', 'step<0.17>_words<2>_length<250>', 'step<0.17>_words<3>_length<150>', 'step<0.17>_words<3>_length<200>', 'step<0.17>_words<3>_length<250>', 'step<0.17>_words<4>_length<150>', 'step<0.17>_words<4>_length<200>', 'step<0.17>_words<4>_length<250>', 'step<0.17>_words<5>_length<150>', 'step<0.17>_words<5>_length<200>', 'step<0.17>_words<5>_length<250>', 'step<0.18>_words<1>_length<150>', 'step<0.18>_words<1>_length<200>', 'step<0.18>_words<1>_length<250>', 'step<0.18>_words<2>_length<150>', 'step<0.18>_words<2>_length<200>', 'step<0.18>_words<2>_length<250>', 'step<0.18>_words<3>_length<150>', 'step<0.18>_words<3>_length<200>', 'step<0.18>_words<3>_length<250>', 'step<0.18>_words<4>_length<150>', 'step<0.18>_words<4>_length<200>', 'step<0.18>_words<4>_length<250>', 'step<0.18>_words<5>_length<150>', 'step<0.18>_words<5>_length<200>', 'step<0.18>_words<5>_length<250>', 'step<0.19>_words<1>_length<150>', 'step<0.19>_words<1>_length<200>', 'step<0.19>_words<1>_length<250>', 'step<0.19>_words<2>_length<150>', 'step<0.19>_words<2>_length<200>', 'step<0.19>_words<2>_length<250>', 'step<0.19>_words<3>_length<150>', 'step<0.19>_words<3>_length<200>', 'step<0.19>_words<3>_length<250>', 'step<0.19>_words<4>_length<150>', 'step<0.19>_words<4>_length<200>', 'step<0.19>_words<4>_length<250>', 'step<0.19>_words<5>_length<150>', 'step<0.19>_words<5>_length<200>', 'step<0.19>_words<5>_length<250>', 'step<0.2>_words<1>_length<150>', 'step<0.2>_words<1>_length<200>', 'step<0.2>_words<1>_length<250>', 'step<0.2>_words<2>_length<150>', 'step<0.2>_words<2>_length<200>', 'step<0.2>_words<2>_length<250>', 'step<0.2>_words<3>_length<150>', 'step<0.2>_words<3>_length<200>', 'step<0.2>_words<3>_length<250>', 'step<0.2>_words<4>_length<150>', 'step<0.2>_words<4>_length<200>', 'step<0.2>_words<4>_length<250>', 'step<0.2>_words<5>_length<150>', 'step<0.2>_words<5>_length<200>', 'step<0.2>_words<5>_length<250>', 'step<0.21>_words<1>_length<150>', 'step<0.21>_words<1>_length<200>', 'step<0.21>_words<1>_length<250>', 'step<0.21>_words<2>_length<150>', 'step<0.21>_words<2>_length<200>', 'step<0.21>_words<2>_length<250>', 'step<0.21>_words<3>_length<150>', 'step<0.21>_words<3>_length<200>', 'step<0.21>_words<3>_length<250>', 'step<0.21>_words<4>_length<150>', 'step<0.21>_words<4>_length<200>', 'step<0.21>_words<4>_length<250>', 'step<0.21>_words<5>_length<150>', 'step<0.21>_words<5>_length<200>', 'step<0.21>_words<5>_length<250>', 'step<0.22>_words<1>_length<150>', 'step<0.22>_words<1>_length<200>', 'step<0.22>_words<1>_length<250>', 'step<0.22>_words<2>_length<150>', 'step<0.22>_words<2>_length<200>', 'step<0.22>_words<2>_length<250>', 'step<0.22>_words<3>_length<150>', 'step<0.22>_words<3>_length<200>', 'step<0.22>_words<3>_length<250>', 'step<0.22>_words<4>_length<150>', 'step<0.22>_words<4>_length<200>', 'step<0.22>_words<4>_length<250>', 'step<0.22>_words<5>_length<150>', 'step<0.22>_words<5>_length<200>', 'step<0.22>_words<5>_length<250>', 'step<0.23>_words<1>_length<150>', 'step<0.23>_words<1>_length<200>', 'step<0.23>_words<1>_length<250>', 'step<0.23>_words<2>_length<150>', 'step<0.23>_words<2>_length<200>', 'step<0.23>_words<2>_length<250>', 'step<0.23>_words<3>_length<150>', 'step<0.23>_words<3>_length<200>', 'step<0.23>_words<3>_length<250>', 'step<0.23>_words<4>_length<150>', 'step<0.23>_words<4>_length<200>', 'step<0.23>_words<4>_length<250>', 'step<0.23>_words<5>_length<150>', 'step<0.23>_words<5>_length<200>', 'step<0.23>_words<5>_length<250>', 'step<0.24>_words<1>_length<150>', 'step<0.24>_words<1>_length<200>', 'step<0.24>_words<1>_length<250>', 'step<0.24>_words<2>_length<150>', 'step<0.24>_words<2>_length<200>', 'step<0.24>_words<2>_length<250>', 'step<0.24>_words<3>_length<150>', 'step<0.24>_words<3>_length<200>', 'step<0.24>_words<3>_length<250>', 'step<0.24>_words<4>_length<150>', 'step<0.24>_words<4>_length<200>', 'step<0.24>_words<4>_length<250>', 'step<0.24>_words<5>_length<150>', 'step<0.24>_words<5>_length<200>', 'step<0.24>_words<5>_length<250>', 'step<0.25>_words<1>_length<150>', 'step<0.25>_words<1>_length<200>', 'step<0.25>_words<1>_length<250>', 'step<0.25>_words<2>_length<150>', 'step<0.25>_words<2>_length<200>', 'step<0.25>_words<2>_length<250>', 'step<0.25>_words<3>_length<150>', 'step<0.25>_words<3>_length<200>', 'step<0.25>_words<3>_length<250>', 'step<0.25>_words<4>_length<150>', 'step<0.25>_words<4>_length<200>', 'step<0.25>_words<4>_length<250>', 'step<0.25>_words<5>_length<150>', 'step<0.25>_words<5>_length<200>', 'step<0.25>_words<5>_length<250>', 'step<0.26>_words<1>_length<150>', 'step<0.26>_words<1>_length<200>', 'step<0.26>_words<1>_length<250>', 'step<0.26>_words<2>_length<150>', 'step<0.26>_words<2>_length<200>', 'step<0.26>_words<2>_length<250>', 'step<0.26>_words<3>_length<150>', 'step<0.26>_words<3>_length<200>', 'step<0.26>_words<3>_length<250>', 'step<0.26>_words<4>_length<150>', 'step<0.26>_words<4>_length<200>', 'step<0.26>_words<4>_length<250>', 'step<0.26>_words<5>_length<150>', 'step<0.26>_words<5>_length<200>', 'step<0.26>_words<5>_length<250>', 'step<0.27>_words<1>_length<150>', 'step<0.27>_words<1>_length<200>', 'step<0.27>_words<1>_length<250>', 'step<0.27>_words<2>_length<150>', 'step<0.27>_words<2>_length<200>', 'step<0.27>_words<2>_length<250>', 'step<0.27>_words<3>_length<150>', 'step<0.27>_words<3>_length<200>', 'step<0.27>_words<3>_length<250>', 'step<0.27>_words<4>_length<150>', 'step<0.27>_words<4>_length<200>', 'step<0.27>_words<4>_length<250>', 'step<0.27>_words<5>_length<150>', 'step<0.27>_words<5>_length<200>', 'step<0.27>_words<5>_length<250>', 'step<0.28>_words<1>_length<150>', 'step<0.28>_words<1>_length<200>', 'step<0.28>_words<1>_length<250>', 'step<0.28>_words<2>_length<150>', 'step<0.28>_words<2>_length<200>', 'step<0.28>_words<2>_length<250>', 'step<0.28>_words<3>_length<150>', 'step<0.28>_words<3>_length<200>', 'step<0.28>_words<3>_length<250>', 'step<0.28>_words<4>_length<150>', 'step<0.28>_words<4>_length<200>', 'step<0.28>_words<4>_length<250>', 'step<0.28>_words<5>_length<150>', 'step<0.28>_words<5>_length<200>', 'step<0.28>_words<5>_length<250>', 'step<0.29>_words<1>_length<150>', 'step<0.29>_words<1>_length<200>', 'step<0.29>_words<1>_length<250>', 'step<0.29>_words<2>_length<150>', 'step<0.29>_words<2>_length<200>', 'step<0.29>_words<2>_length<250>', 'step<0.29>_words<3>_length<150>', 'step<0.29>_words<3>_length<200>', 'step<0.29>_words<3>_length<250>', 'step<0.29>_words<4>_length<150>', 'step<0.29>_words<4>_length<200>', 'step<0.29>_words<4>_length<250>', 'step<0.29>_words<5>_length<150>', 'step<0.29>_words<5>_length<200>', 'step<0.29>_words<5>_length<250>', 'step<0.3>_words<1>_length<150>', 'step<0.3>_words<1>_length<200>', 'step<0.3>_words<1>_length<250>', 'step<0.3>_words<2>_length<150>', 'step<0.3>_words<2>_length<200>', 'step<0.3>_words<2>_length<250>', 'step<0.3>_words<3>_length<150>', 'step<0.3>_words<3>_length<200>', 'step<0.3>_words<3>_length<250>', 'step<0.3>_words<4>_length<150>', 'step<0.3>_words<4>_length<200>', 'step<0.3>_words<4>_length<250>', 'step<0.3>_words<5>_length<150>', 'step<0.3>_words<5>_length<200>', 'step<0.3>_words<5>_length<250>', 'step<0.31>_words<1>_length<150>', 'step<0.31>_words<1>_length<200>', 'step<0.31>_words<1>_length<250>', 'step<0.31>_words<2>_length<150>', 'step<0.31>_words<2>_length<200>', 'step<0.31>_words<2>_length<250>', 'step<0.31>_words<3>_length<150>', 'step<0.31>_words<3>_length<200>', 'step<0.31>_words<3>_length<250>', 'step<0.31>_words<4>_length<150>', 'step<0.31>_words<4>_length<200>', 'step<0.31>_words<4>_length<250>', 'step<0.31>_words<5>_length<150>', 'step<0.31>_words<5>_length<200>', 'step<0.31>_words<5>_length<250>', 'step<0.32>_words<1>_length<150>', 'step<0.32>_words<1>_length<200>', 'step<0.32>_words<1>_length<250>', 'step<0.32>_words<2>_length<150>', 'step<0.32>_words<2>_length<200>', 'step<0.32>_words<2>_length<250>', 'step<0.32>_words<3>_length<150>', 'step<0.32>_words<3>_length<200>', 'step<0.32>_words<3>_length<250>', 'step<0.32>_words<4>_length<150>', 'step<0.32>_words<4>_length<200>', 'step<0.32>_words<4>_length<250>', 'step<0.32>_words<5>_length<150>', 'step<0.32>_words<5>_length<200>', 'step<0.32>_words<5>_length<250>', 'step<0.33>_words<1>_length<150>', 'step<0.33>_words<1>_length<200>', 'step<0.33>_words<1>_length<250>', 'step<0.33>_words<2>_length<150>', 'step<0.33>_words<2>_length<200>', 'step<0.33>_words<2>_length<250>', 'step<0.33>_words<3>_length<150>', 'step<0.33>_words<3>_length<200>', 'step<0.33>_words<3>_length<250>', 'step<0.33>_words<4>_length<150>', 'step<0.33>_words<4>_length<200>', 'step<0.33>_words<4>_length<250>', 'step<0.33>_words<5>_length<150>', 'step<0.33>_words<5>_length<200>', 'step<0.33>_words<5>_length<250>', 'step<0.34>_words<1>_length<150>', 'step<0.34>_words<1>_length<200>', 'step<0.34>_words<1>_length<250>', 'step<0.34>_words<2>_length<150>', 'step<0.34>_words<2>_length<200>', 'step<0.34>_words<2>_length<250>', 'step<0.34>_words<3>_length<150>', 'step<0.34>_words<3>_length<200>', 'step<0.34>_words<3>_length<250>', 'step<0.34>_words<4>_length<150>', 'step<0.34>_words<4>_length<200>', 'step<0.34>_words<4>_length<250>', 'step<0.34>_words<5>_length<150>', 'step<0.34>_words<5>_length<200>', 'step<0.34>_words<5>_length<250>', 'step<0.35>_words<1>_length<150>', 'step<0.35>_words<1>_length<200>', 'step<0.35>_words<1>_length<250>', 'step<0.35>_words<2>_length<150>', 'step<0.35>_words<2>_length<200>', 'step<0.35>_words<2>_length<250>', 'step<0.35>_words<3>_length<150>', 'step<0.35>_words<3>_length<200>', 'step<0.35>_words<3>_length<250>', 'step<0.35>_words<4>_length<150>', 'step<0.35>_words<4>_length<200>', 'step<0.35>_words<4>_length<250>', 'step<0.35>_words<5>_length<150>', 'step<0.35>_words<5>_length<200>', 'step<0.35>_words<5>_length<250>', 'step<0.36>_words<1>_length<150>', 'step<0.36>_words<1>_length<200>', 'step<0.36>_words<1>_length<250>', 'step<0.36>_words<2>_length<150>', 'step<0.36>_words<2>_length<200>', 'step<0.36>_words<2>_length<250>', 'step<0.36>_words<3>_length<150>', 'step<0.36>_words<3>_length<200>', 'step<0.36>_words<3>_length<250>', 'step<0.36>_words<4>_length<150>', 'step<0.36>_words<4>_length<200>', 'step<0.36>_words<4>_length<250>', 'step<0.36>_words<5>_length<150>', 'step<0.36>_words<5>_length<200>', 'step<0.36>_words<5>_length<250>', 'step<0.37>_words<1>_length<150>', 'step<0.37>_words<1>_length<200>', 'step<0.37>_words<1>_length<250>', 'step<0.37>_words<2>_length<150>', 'step<0.37>_words<2>_length<200>', 'step<0.37>_words<2>_length<250>', 'step<0.37>_words<3>_length<150>', 'step<0.37>_words<3>_length<200>', 'step<0.37>_words<3>_length<250>', 'step<0.37>_words<4>_length<150>', 'step<0.37>_words<4>_length<200>', 'step<0.37>_words<4>_length<250>', 'step<0.37>_words<5>_length<150>', 'step<0.37>_words<5>_length<200>', 'step<0.37>_words<5>_length<250>', 'step<0.38>_words<1>_length<150>', 'step<0.38>_words<1>_length<200>', 'step<0.38>_words<1>_length<250>', 'step<0.38>_words<2>_length<150>', 'step<0.38>_words<2>_length<200>', 'step<0.38>_words<2>_length<250>', 'step<0.38>_words<3>_length<150>', 'step<0.38>_words<3>_length<200>', 'step<0.38>_words<3>_length<250>', 'step<0.38>_words<4>_length<150>', 'step<0.38>_words<4>_length<200>', 'step<0.38>_words<4>_length<250>', 'step<0.38>_words<5>_length<150>', 'step<0.38>_words<5>_length<200>', 'step<0.38>_words<5>_length<250>', 'step<0.39>_words<1>_length<150>', 'step<0.39>_words<1>_length<200>', 'step<0.39>_words<1>_length<250>', 'step<0.39>_words<2>_length<150>', 'step<0.39>_words<2>_length<200>', 'step<0.39>_words<2>_length<250>', 'step<0.39>_words<3>_length<150>', 'step<0.39>_words<3>_length<200>', 'step<0.39>_words<3>_length<250>', 'step<0.39>_words<4>_length<150>', 'step<0.39>_words<4>_length<200>', 'step<0.39>_words<4>_length<250>', 'step<0.39>_words<5>_length<150>', 'step<0.39>_words<5>_length<200>', 'step<0.39>_words<5>_length<250>', 'step<0.4>_words<1>_length<150>', 'step<0.4>_words<1>_length<200>', 'step<0.4>_words<1>_length<250>', 'step<0.4>_words<2>_length<150>', 'step<0.4>_words<2>_length<200>', 'step<0.4>_words<2>_length<250>', 'step<0.4>_words<3>_length<150>', 'step<0.4>_words<3>_length<200>', 'step<0.4>_words<3>_length<250>', 'step<0.4>_words<4>_length<150>', 'step<0.4>_words<4>_length<200>', 'step<0.4>_words<4>_length<250>', 'step<0.4>_words<5>_length<150>', 'step<0.4>_words<5>_length<200>', 'step<0.4>_words<5>_length<250>', 'step<0.41>_words<1>_length<150>', 'step<0.41>_words<1>_length<200>', 'step<0.41>_words<1>_length<250>', 'step<0.41>_words<2>_length<150>', 'step<0.41>_words<2>_length<200>', 'step<0.41>_words<2>_length<250>', 'step<0.41>_words<3>_length<150>', 'step<0.41>_words<3>_length<200>', 'step<0.41>_words<3>_length<250>', 'step<0.41>_words<4>_length<150>', 'step<0.41>_words<4>_length<200>', 'step<0.41>_words<4>_length<250>', 'step<0.41>_words<5>_length<150>', 'step<0.41>_words<5>_length<200>', 'step<0.41>_words<5>_length<250>', 'step<0.42>_words<1>_length<150>', 'step<0.42>_words<1>_length<200>', 'step<0.42>_words<1>_length<250>', 'step<0.42>_words<2>_length<150>', 'step<0.42>_words<2>_length<200>', 'step<0.42>_words<2>_length<250>', 'step<0.42>_words<3>_length<150>', 'step<0.42>_words<3>_length<200>', 'step<0.42>_words<3>_length<250>', 'step<0.42>_words<4>_length<150>', 'step<0.42>_words<4>_length<200>', 'step<0.42>_words<4>_length<250>', 'step<0.42>_words<5>_length<150>', 'step<0.42>_words<5>_length<200>', 'step<0.42>_words<5>_length<250>', 'step<0.43>_words<1>_length<150>', 'step<0.43>_words<1>_length<200>', 'step<0.43>_words<1>_length<250>', 'step<0.43>_words<2>_length<150>', 'step<0.43>_words<2>_length<200>', 'step<0.43>_words<2>_length<250>', 'step<0.43>_words<3>_length<150>', 'step<0.43>_words<3>_length<200>', 'step<0.43>_words<3>_length<250>', 'step<0.43>_words<4>_length<150>', 'step<0.43>_words<4>_length<200>', 'step<0.43>_words<4>_length<250>', 'step<0.43>_words<5>_length<150>', 'step<0.43>_words<5>_length<200>', 'step<0.43>_words<5>_length<250>', 'step<0.44>_words<1>_length<150>', 'step<0.44>_words<1>_length<200>', 'step<0.44>_words<1>_length<250>', 'step<0.44>_words<2>_length<150>', 'step<0.44>_words<2>_length<200>', 'step<0.44>_words<2>_length<250>', 'step<0.44>_words<3>_length<150>', 'step<0.44>_words<3>_length<200>', 'step<0.44>_words<3>_length<250>', 'step<0.44>_words<4>_length<150>', 'step<0.44>_words<4>_length<200>', 'step<0.44>_words<4>_length<250>', 'step<0.44>_words<5>_length<150>', 'step<0.44>_words<5>_length<200>', 'step<0.44>_words<5>_length<250>', 'step<0.45>_words<1>_length<150>', 'step<0.45>_words<1>_length<200>', 'step<0.45>_words<1>_length<250>', 'step<0.45>_words<2>_length<150>', 'step<0.45>_words<2>_length<200>', 'step<0.45>_words<2>_length<250>', 'step<0.45>_words<3>_length<150>', 'step<0.45>_words<3>_length<200>', 'step<0.45>_words<3>_length<250>', 'step<0.45>_words<4>_length<150>', 'step<0.45>_words<4>_length<200>', 'step<0.45>_words<4>_length<250>', 'step<0.45>_words<5>_length<150>', 'step<0.45>_words<5>_length<200>', 'step<0.45>_words<5>_length<250>', 'step<0.46>_words<1>_length<150>', 'step<0.46>_words<1>_length<200>', 'step<0.46>_words<1>_length<250>', 'step<0.46>_words<2>_length<150>', 'step<0.46>_words<2>_length<200>', 'step<0.46>_words<2>_length<250>', 'step<0.46>_words<3>_length<150>', 'step<0.46>_words<3>_length<200>', 'step<0.46>_words<3>_length<250>', 'step<0.46>_words<4>_length<150>', 'step<0.46>_words<4>_length<200>', 'step<0.46>_words<4>_length<250>', 'step<0.46>_words<5>_length<150>', 'step<0.46>_words<5>_length<200>', 'step<0.46>_words<5>_length<250>', 'step<0.47>_words<1>_length<150>', 'step<0.47>_words<1>_length<200>', 'step<0.47>_words<1>_length<250>', 'step<0.47>_words<2>_length<150>', 'step<0.47>_words<2>_length<200>', 'step<0.47>_words<2>_length<250>', 'step<0.47>_words<3>_length<150>', 'step<0.47>_words<3>_length<200>', 'step<0.47>_words<3>_length<250>', 'step<0.47>_words<4>_length<150>', 'step<0.47>_words<4>_length<200>', 'step<0.47>_words<4>_length<250>', 'step<0.47>_words<5>_length<150>', 'step<0.47>_words<5>_length<200>', 'step<0.47>_words<5>_length<250>', 'step<0.48>_words<1>_length<150>', 'step<0.48>_words<1>_length<200>', 'step<0.48>_words<1>_length<250>', 'step<0.48>_words<2>_length<150>', 'step<0.48>_words<2>_length<200>', 'step<0.48>_words<2>_length<250>', 'step<0.48>_words<3>_length<150>', 'step<0.48>_words<3>_length<200>', 'step<0.48>_words<3>_length<250>', 'step<0.48>_words<4>_length<150>', 'step<0.48>_words<4>_length<200>', 'step<0.48>_words<4>_length<250>', 'step<0.48>_words<5>_length<150>', 'step<0.48>_words<5>_length<200>', 'step<0.48>_words<5>_length<250>', 'step<0.49>_words<1>_length<150>', 'step<0.49>_words<1>_length<200>', 'step<0.49>_words<1>_length<250>', 'step<0.49>_words<2>_length<150>', 'step<0.49>_words<2>_length<200>', 'step<0.49>_words<2>_length<250>', 'step<0.49>_words<3>_length<150>', 'step<0.49>_words<3>_length<200>', 'step<0.49>_words<3>_length<250>', 'step<0.49>_words<4>_length<150>', 'step<0.49>_words<4>_length<200>', 'step<0.49>_words<4>_length<250>', 'step<0.49>_words<5>_length<150>', 'step<0.49>_words<5>_length<200>', 'step<0.49>_words<5>_length<250>']

In [33]:
import tqdm

In [34]:
def toxic_words_detector(toxic_words, comment):
    for w in toxic_words:
        if w in comment:
            return True
    return False

In [35]:
for i in range(1, len(step_roc_dict), 80):
    #print(i)
    m = re.search('step<(.+?)>_words<(.+?)>_length<(.+?)>', list(step_roc_dict)[i])
    step, words, length = m.group(1), m.group(2), m.group(3)
    
    #print(step, words, length)
    sub_w_rule = submission.merge(test[['id','comment_text_cleaned','word_count']],on='id')
    toxic_words = toxic_words_poll_reordered[:int(words)+1]
    
    sub_w_rule['toxic_word_exist'] = sub_w_rule['comment_text_cleaned'].apply(lambda x: toxic_words_detector(toxic_words, x))            
    #print('{} \t comments containing any toxic words.'.format(sum(sub_w_rule['toxic_word_exist'])))
    
    sub_w_rule['toxic'] = sub_w_rule.apply(lambda r: r.toxic+float(step) if r.toxic_word_exist and r.word_count < int(length) else r.toxic, axis=1)
    
    sub_final = sub_w_rule.drop(['comment_text_cleaned','toxic_word_exist','word_count'], axis=1)
    assert sub_final.shape == submission.shape
    
    import time
    BUILD_ID = '{}_{}_{}_{}'.format(step, words, length, int(time.time()))
    print(BUILD_ID)
    sub_final.to_csv('/home/kai/data/shiyi/toxic/submissions/sub_FastText_w_rule_' + str(BUILD_ID) + '.csv', index=False)

0.01_1_200_1518810194
0.06_3_150_1518810199
0.11_4_250_1518810203
0.17_1_200_1518810207
0.22_3_150_1518810211
0.27_4_250_1518810214
0.33_1_200_1518810218
0.38_3_150_1518810222
0.43_4_250_1518810226
0.49_1_200_1518810230


In [36]:
sum(sub_w_rule.toxic>0.9), sum(sub_w_rule.toxic<0.1)

(16711, 108272)

In [37]:
sum(submission.toxic>0.9), sum(submission.toxic<0.1)

(16694, 108272)

In [39]:
def get_lstm_model(window_length, n_features, lstm_units, dense_units, label_cols, dropout, load_model=False, load_model_file=None):
    #embedding_matrix = get_embedding_matrix(embedding_file, embed_size, max_features, tokenizer)
    #input = Input(shape=(max_len, ))
    input = Input(shape=(window_length, n_features))
    #x = Embedding(max_features, embed_size, weights=[embedding_matrix])(input)
    x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout))(input)
    x = GlobalMaxPool1D()(x)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(len(label_cols), activation='sigmoid')(x)
    model = Model(inputs=input, outputs=x)
    
    #pdb.set_trace()
    if (load_model):
        model.load_weights(load_model_file)
        
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [42]:
DROPOUT = 0.1
BATCH_SIZE = 32    
LSTM_UNITS = 50
DENSE_UNITS = 50
window_length = 200
n_features = 300

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

SAVE_DIR = '/home/kai/data/shiyi/toxic/'
SUB_DIR = SAVE_DIR + 'submissions/'
print(SUB_DIR)

print('loading saved model')
LOAD_MODEL_FILE = SAVE_DIR + 'models/mod_FastText_cleaned_200_300_50_50_0.1_32_4_1518806221.hdf5'
model = get_lstm_model(window_length, n_features, LSTM_UNITS, DENSE_UNITS, label_cols, DROPOUT,
                       load_model=True, load_model_file=LOAD_MODEL_FILE)

/home/kai/data/shiyi/toxic/submissions/
loading saved model


In [47]:
x_val = np.load('/home/kai/data/shiyi/toxic/saved_files/x_val_np.npy')
y_val = np.load('/home/kai/data/shiyi/toxic/saved_files/y_val_np.npy')
x_val.shape, y_val.shape

((15957, 200, 300), (15957, 6))

In [49]:
y_val_pred = model.predict(x_val, verbose=1)

15957/15957 [==============================] - 47s 3ms/step


In [50]:
y_val_pred.shape

(15957, 6)

In [54]:
roc_auc_score(y_val, y_val_pred)

0.989925306112231

In [91]:
'aaa %-100s {0: <16}'.format('Hi', '<15')

'aaa %-100s Hi              '

In [86]:
'{message: <{fill}}'.format(message='Hi', fill='16')

'Hi              '

In [96]:
mean_auc = 0
for i in range(len(label_cols)):
    fpr, tpr, thresholds = roc_curve(y_val[:,i], y_val_pred[:,i], pos_label=1.0)
    auc_temp = auc(fpr, tpr)
    print('Label: {:20} Threashold count: {} \t AUC: {}'.format(label_cols[i], len(thresholds), auc_temp))
    mean_auc += auc_temp
mean_auc/=len(label_cols)
print('Mean AUC of all classes: {}'.format(mean_auc))
assert mean_auc == roc_auc_score(y_val, y_val_pred)

Label: toxic                Threashold count: 863 	 AUC: 0.9836142374721919
Label: severe_toxic         Threashold count: 265 	 AUC: 0.9916066721494696
Label: obscene              Threashold count: 520 	 AUC: 0.9926648835859645
Label: threat               Threashold count: 153 	 AUC: 0.9935311498082606
Label: insult               Threashold count: 657 	 AUC: 0.9866579003959778
Label: identity_hate        Threashold count: 245 	 AUC: 0.9914769932615218
Mean AUC of all classes: 0.989925306112231


In [104]:
individual_label_auc_report = ''
for i in range(len(label_cols)):
    fpr, tpr, thresholds = roc_curve(y_val[:,i], y_val_pred[:,i], pos_label=1.0)
    auc_temp = auc(fpr, tpr)
    individual_label_auc_report += 'Label: {:20} Threashold count: {} \t AUC: {}\n'.format(label_cols[i], len(thresholds), auc_temp)

In [106]:
print(individual_label_auc_report)

Label: toxic                Threashold count: 863 	 AUC: 0.9836142374721919
Label: severe_toxic         Threashold count: 265 	 AUC: 0.9916066721494696
Label: obscene              Threashold count: 520 	 AUC: 0.9926648835859645
Label: threat               Threashold count: 153 	 AUC: 0.9935311498082606
Label: insult               Threashold count: 657 	 AUC: 0.9866579003959778
Label: identity_hate        Threashold count: 245 	 AUC: 0.9914769932615218



In [103]:
for i in range(len(label_cols)):
    print(np.sum(y_val[:,i]==1))
np.corrcoef([863,265,520,153,657,245],[1543,150,864,50,817,144]) # threashold count and label=1 data count
# this shows the threashold count is proportional to the label=1 data count

1543
150
864
50
817
144


array([[1.        , 0.97760483],
       [0.97760483, 1.        ]])

In [100]:
np.corrcoef([863,265,520,153,657,245],[1543,150,864,50,817,144])

array([[1.        , 0.97760483],
       [0.97760483, 1.        ]])

In [70]:
fpr.shape, tpr.shape, thresholds.shape

((863,), (863,), (863,))

In [61]:
len(thresholds), thresholds[0:5], thresholds[-5:]

(863, array([0.9999957 , 0.9999329 , 0.99993193, 0.9998716 , 0.9998677 ],
       dtype=float32), array([4.4010125e-05, 4.3965534e-05, 2.3929031e-05, 2.3893048e-05,
        5.2790972e-07], dtype=float32))

In [ ]:
model

In [113]:
def evaluate(model, file_path, x_val, y_val, label_cols, metrics='ROC'):
    model.load_weights(file_path)
    y_val_preds = model.predict(x_val)
    individual_label_auc_report = ''
    for i in range(len(label_cols)):
        fpr, tpr, thresholds = roc_curve(y_val[:,i], y_val_pred[:,i], pos_label=1.0)
        auc_temp = auc(fpr, tpr)
        individual_label_auc_report += '\nLabel: {:20} Threashold count: {} \t AUC: {}'.format(label_cols[i], len(thresholds), auc_temp)
        #mean_auc += auc_temp # this can be computed by roc_auc_score
    #mean_auc/=len(label_cols)
    individual_label_auc_report+='\n'
    if metrics=='ROC':
        return roc_auc_score(y_val, y_val_preds), individual_label_auc_report
    raise ValueError('The chosen metrics is not implemented yet')

In [114]:
roc, report = evaluate(model, LOAD_MODEL_FILE, x_val, y_val, label_cols)

In [115]:
print('ROC on each label: {}'.format(report))
print('ROC: {}'.format(roc))

ROC on each label: 
Label: toxic                Threashold count: 863 	 AUC: 0.9836142374721919
Label: severe_toxic         Threashold count: 265 	 AUC: 0.9916066721494696
Label: obscene              Threashold count: 520 	 AUC: 0.9926648835859645
Label: threat               Threashold count: 153 	 AUC: 0.9935311498082606
Label: insult               Threashold count: 657 	 AUC: 0.9866579003959778
Label: identity_hate        Threashold count: 245 	 AUC: 0.9914769932615218

ROC: 0.989925306112231
